# Trouver et appliquer les règles à entropie nulle


## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [103]:
# -*- coding: utf8 -*-
import codecs,glob
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import networkx as nx
import pickle,yaml
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

### Préparation des matrices de traits

In [104]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [105]:
ipaIn = listerUnicode(u"SZNêôârEHO")
ipaOut= [u"ʃ",u"ʒ",u"ŋ",u"ɛ̃",u"ɔ̃",u"ɑ̃",u"ʁ",u"ɛ",u"ɥ",u"ɔ"]
toipa = dict(zip(ipaIn, ipaOut))

S Z N ê ô â r E H O


In [106]:
import unicodedata
def listerUnicode(chaine):
    result=[]
    comb=False
    prec=u""
    for char in chaine:
        print char,
        if unicodedata.combining(char):
            result.append(prec+char)
            prec=""
        else:
            result.append(prec)
            prec=char
    result.append(prec)
    return [r for r in result if r!=""]

def coderIPA(chaine,table=toipa):
    result=chaine
    for k in table:
        result=result.replace(k,table[k])
    return result

In [107]:
print coderIPA(u"SâZ piNpôN")

ʃɑ̃ʒ piŋpɔ̃ŋ


In [142]:
rep="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Longitudinales/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/StemSpaces/LongitudinalesRnd/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/StemSpaces/Longitudinales/"
fichiers=glob.glob(rep+"*X-Regles.pkl")
samples=[f.rsplit("/",1)[-1].split("Regles")[0] for f in fichiers]
samples={int(s.split("-")[1]):s for s in samples}
samples

{1: 'Longitudinal-01-T1000000-F39898-X-',
 2: 'Longitudinal-02-T1000000-F39898-X-',
 54: 'Longitudinal-54-T1000000-F39898-X-'}

In [109]:
sample=samples[1]
fRulesPMS="Regles.pkl"
fRulesPMO="Morphomes-Regles.pkl"

# Classe pour la gestion des patrons, des classes et des transformations

In [110]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print "le numéro de forme n'est pas dans [1,2]",n

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=False):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
#                if debug: 
#                    print (forme, file=logfile)
#                    print ("pas de classe",idClasseForme, file=logfile)
#                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
#        else:
#            if debug:
#                print (forme, file=logfile) 
#                print ("pas de patron", file=logfile)
        return sortieForme
        

## Ouvrir les fichiers de règles

In [111]:
def openRules(numero,typeRules="PMS"):
    rules=None
    if typeRules=="PMS" and numero in samples:
        with open(rep+samples[numero]+fRulesPMS, 'rb') as input:
            rules = pickle.load(input)
    elif typeRules=="PMO" and numero in samples:
        with open(rep+samples[numero]+fRulesPMO, 'rb') as input:
            rules = pickle.load(input)
    return rules

In [209]:
numero=2
typeRules="PMO"
avecContexte=True

In [211]:
rules=openRules(numero,typeRules=typeRules)
rules[(u'pi2S', u'ai2P')]

<__main__.classesPaire instance at 0x7fb9941e6e60>

In [114]:
rules[("ii1S","ii2P")].patrons

{u'E-e': u'^(.*[ptkbdgfsSvzZjlrwH][iyEe92a][Jj])E$',
 u'E-ije': u'^(.*[ptkbdgfsSvzZr][lrE6])E$',
 u'E-je': u'^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZmnJjlr])E$',
 u'HE-je': u'^(.*)yHE$',
 u'wE-je': u'^(.*[ptkbdgfsSvzZmnJlr])uwE$'}

In [115]:
zeroEntropie=[]
for paire in rules:
    classe=rules[paire].classe
    c1,c2=paire
    if c1!=c2:
        zero=True
        print paire
        for transformation in classe:
            choix=classe[transformation]
            if len(choix)>1:
                print choix
                zero=False
        if zero:
            print "zéro",choix
            zeroEntropie.append(paire)
        print

(u'ps2P', u'is1S')
zéro {u'swaje-fys': 1}

(u'ii3S', u'ps3P')
{u'E-': 2, u'EjE-wa': 2, u'jE-': 1}
{u'E-': 2, u'lE-j': 7}
{u'E-': 338, u'O.E-o.': 62}
{u'6.E-E.': 1, u'6nE-jEn': 26}
{u'E-': 8, u'avE-E': 2}
{u'6.E-9.': 2, u'6.E-E.': 3}
{u'E-': 118, u'jE-': 90}
{u'E-': 1, u'u.E-9.': 2}
{u'E-': 12, u'jE-i': 164}

(u'pc2S', u'ai2P')
zéro {u'6rE-it': 4}

(u'is2S', u'pc3P')
{u'ys-6vrE': 8, u'ys-OrE': 1}
{u'as-6rE': 2, u'alas-irE': 3}
{u's-rE': 3, u'is-rE': 33}

(u'ps2S', u'ii2P')
{u'-ije': 196, u'-je': 1}
{u'-je': 149, u'E.-6.je': 97}
{u'-e': 15, u'-je': 14}
{u'-e': 6, u'j-lje': 4}
{u'-je': 18, u'E-avje': 2}

(u'pc2P', u'ai2P')
zéro {u's6rje-fyt': 1}

(u'ii3P', u'pi2P')
{u'E-e': 104, u'zE-t': 14}
{u'E-e': 3, u'EjE-waje': 2}

(u'fi3P', u'pi2P')
{u'6r\xf4-e': 148, u'E.6r\xf4-6.e': 104}
{u'r\xf4-je': 2, u'Er\xf4-waje': 1}
{u'r\xf4-e': 1, u'\xeadr\xf4-EJe': 20}
{u'ir\xf4-e': 12, u'ir\xf4-je': 23, u'r\xf4-se': 13}
{u'r\xf4-ze': 4, u'r\xf4-je': 5}
{u'ir\xf4-e': 1, u'ir\xf4-je': 5, u'r\xf4-se': 10}
{

(u'ii1S', u'is3S')
{u'E-a': 11, u'avE-y': 2}
{u'E-a': 125, u'E-y': 9}
{u'izE-y': 3, u'E-a': 79}
{u'E-a': 94, u'E-i': 18, u'zE-': 14}
{u'E-a': 30, u'EsE-y': 9}
{u'wajE-i': 6, u'E-a': 8, u'wajE-y': 1}
{u'ivE-Eky': 3, u'E-a': 1}
{u'E-a': 1, u'6nE-\xea': 26}
{u'sE-': 246, u'E-a': 22}
{u'jE-i': 1, u'E-a': 14}
{u'E-a': 160, u'E-y': 5}
{u'EsE-aki': 2, u'EsE-y': 3}
{u'E-a': 2718, u'E-i': 140}
{u'E-a': 7, u'wajE-y': 6}
{u'E-a': 1, u'EjE-i': 2}

(u'pc2P', u'ppFS')
{u'rje-': 67, u'irje-ale': 1}
{u'6rje-Et': 5, u'6rje-e': 34}
{u'rje-': 147, u'rje-t': 15}

(u'ai1P', u'fi1P')
{u'm-r\xf4': 37, u'im-r\xf4': 31}
{u'am-6r\xf4': 7, u'jam-r\xf4': 15}
{u'im-r\xf4': 3, u'im-Er\xf4': 3}
{u'm-r\xf4': 1, u'im-Er\xf4': 2}
{u'6.am-E.6r\xf4': 99, u'am-6r\xf4': 3}
{u'im-6r\xf4': 4, u'm-r\xf4': 4}
{u'am-6r\xf4': 36, u'alam-ir\xf4': 3}
{u'am-6r\xf4': 1, u'wajam-Er\xf4': 8}
{u'ym-6vr\xf4': 8, u'ym-Or\xf4': 1}
{u'im-Etr\xf4': 16, u'm-r\xf4': 2}

(u'ii1S', u'ii3S')
zéro {'-': 4545}

(u'pc2P', u'fi3S')
{u'je-a': 19, u'r

{u'6rE-i': 1, u'6rE-a': 1}
{u'rE-i': 1, u'\xeadrE-EJi': 19}
{u'rE-ja': 11, u'warE-y': 8}
{u'ErE-y': 1, u'rE-ja': 1}
{u'rE-vi': 10, u'rE-ja': 4}
{u'ErE-waja': 7, u'ErE-i': 3}
{u'irE-ja': 8, u'rE-': 35}
{u'6rE-a': 31, u'6rE-i': 5}
{u'rE-ja': 2, u'rE-zi': 18, u'rE-': 5}
{u'irE-ja': 145, u'rE-ja': 5, u'rE-': 213}

(u'fi3P', u'ii1S')
{u'r\xf4-jE': 3, u'Er\xf4-wajE': 14}
{u'ir\xf4-E': 1, u'ir\xf4-jE': 5, u'r\xf4-sE': 10}
{u'r\xf4-wE': 12, u'r\xf4-vE': 2}
{u'ir\xf4-E': 3, u'r\xf4-sE': 2}
{u'6r\xf4-E': 124, u'E.6r\xf4-6.E': 88}
{u'ir\xf4-E': 17, u'r\xf4-zE': 5, u'ir\xf4-jE': 20, u'r\xf4-sE': 63}
{u'Odr\xf4-alE': 4, u'r\xf4-E': 1}
{u'r\xf4-zE': 3, u'r\xf4-jE': 9, u'r\xf4-vE': 10, u'r\xf4-sE': 26, u'ir\xf4-jE': 82}
{u'6r\xf4-E': 27, u'E.6r\xf4-6.E': 16}
{u'r\xf4-zE': 7, u'ir\xf4-jE': 30, u'r\xf4-sE': 81, u'r\xf4-jE': 10}
{u'r\xf4-E': 16, u'tr\xf4-sE': 13}
{u'r\xf4-zE': 4, u'r\xf4-jE': 15}
{u'ir\xf4-jE': 3, u'r\xf4-sE': 18}
{u'ir\xf4-E': 15, u'ir\xf4-jE': 25, u'r\xf4-sE': 22}
{u'r\xf4-E': 1, u'\x

{u'-ij\xf4': 106, u'-j\xf4': 1}

(u'pi3P', u'pi1P')
{u'i-j\xf4': 157, u'-\xf4': 19}
{u'-j\xf4': 17, u'-\xf4': 4}
{u'9.-6.\xf4': 1, u'9.-u.\xf4': 3}
{u'-\xf4': 1, u'Ej-waj\xf4': 1}
{u'-\xf4': 188, u'E.-6.\xf4': 119}
{u'-j\xf4': 2, u'wa-Ej\xf4': 2}

(u'ppFP', u'ppMS')
zéro {'-': 4384}

(u'ppMP', u'fi2S')
{u'e-6ra': 11, u'6.e-E.6ra': 93}
{u'y-ura': 1, u'y-wara': 1}
{u'y-Era': 3, u'y-6vra': 1, u'-ra': 3, u'y-ira': 3}
{u'y-watra': 2, u'y-wara': 1}
{u'y-Era': 4, u'y-ra': 3, u'y-wara': 1}
{u'y-Era': 1, u'y-Ora': 2, u'y-6vra': 1}
{u'e-6ra': 130, u'je-ra': 51}
{u'i-Etra': 16, u'-ra': 7}
{u'-ra': 2, u'i-6ra': 4}
{u'e-Etra': 1, u'e-6ra': 2, u'6.e-E.6ra': 14}
{u'e-6ra': 40, u'ale-ira': 3}
{u'y-6vra': 7, u'y-ra': 51}
{u'je-ira': 164, u'e-6ra': 12}
{u'y-ra': 10, u'y-wara': 1}
{u'i-jEra': 2, u'i-wara': 2}
{u'e-6ra': 1, u'waje-Era': 8, u'je-ra': 2}

(u'is2P', u'ii1S')
zéro {u'fysje-EtE': 1}

(u'ps3S', u'fi2P')
{u'-re': 3, u'wa-Ere': 13}
{u'z-re': 1, u'-6re': 7}
{u'-6re': 30, u's-tre': 9}
{u'-ire': 4, 

zéro {u'sje-m': 9}

(u'pi3S', u'pc1P')
{u'-rj\xf4': 1, u'-vri\xf4': 2}
{u'-dri\xf4': 7, u'-6rj\xf4': 206, u'-tirj\xf4': 2}
{u'-6rj\xf4': 1, u'-irj\xf4': 1}
{u'-6rj\xf4': 27, u'-tirj\xf4': 5}
{u'-tri\xf4': 7, u'-rj\xf4': 5}
{u'-tri\xf4': 20, u'-rj\xf4': 109}
{u'-dri\xf4': 1, u'-6rj\xf4': 113, u'-rj\xf4': 1}
{u'-6rj\xf4': 25, u'-irj\xf4': 1}
{u'-tri\xf4': 7, u'a-Orj\xf4': 1}
{u'-dri\xf4': 13, u'-tirj\xf4': 3}

(u'pc2P', u'pI2P')
{u'6rje-Et': 5, u'6rje-e': 34}
{u'drije-le': 1, u'udrije-6je': 1}
{u'rje-se': 60, u'rje-t': 3}
{u'rje-se': 10, u'irje-e': 10}
{u'rje-se': 56, u'irje-ale': 1}
{u'rje-se': 65, u'rje-t': 4, u'irje-e': 21}

(u'inf', u'pc3P')
{u'waje-ErE': 8, u'je-rE': 2, u'e-6rE': 1}
{u'ir-rE': 9, u'-E': 1}
{u'ale-irE': 3, u'e-6rE': 40}
{u'je-irE': 164, u'e-6rE': 12}
{u'-E': 1, u'war-ErE': 5}
{u'6.e-E.6rE': 107, u'e-6rE': 6}
{u'-E': 3, u'i.-6.E': 4}
{u'je-rE': 52, u'e-6rE': 136}

(u'ppMS', u'ii3S')
{u'y-izE': 2, u'y-EzE': 1, u'y-6vE': 1}
{u'i-jE': 2, u'-sE': 33, u'-vE': 10}
{u'y-izE'

{u'E.6rE-6.E': 57, u'6rE-E': 46}

(u'ps2S', u'pI2S')
{u'wa-je': 2, '-': 2}
{'-': 24, u'-j': 5}
{u'Ej-je': 2, u'Ej-wa': 2}
{u't-': 35, '-': 137}
{u'aj-va': 3, '-': 52}
{u'd-': 41, '-': 34}
{'-': 6, u'v-': 25}
{'-': 54, u's-': 221}
{u'z-': 47, '-': 141}

(u'ii3P', u'ps3S')
{u'E-': 2, u'EjE-wa': 2, u'jE-': 1}
{u'E-': 2, u'lE-j': 7}
{u'E-': 338, u'O.E-o.': 62}
{u'6.E-E.': 1, u'6nE-jEn': 26}
{u'E-': 8, u'avE-E': 2}
{u'6.E-9.': 2, u'6.E-E.': 3}
{u'E-': 118, u'jE-': 90}
{u'E-': 1, u'u.E-9.': 2}
{u'E-': 12, u'jE-i': 164}

(u'fi2P', u'pc2P')
{u'e-je': 10, u'j6re-rje': 8}

(u'is1P', u'pc3S')
zéro {u'ysj\xf4-OrE': 2}

(u'pI2S', u'inf')
{u'-e': 200, u'-dr': 7, u'-tir': 2}
{u'E-air': 1, u'E-avwar': 1}
{u'-e': 230, u'-dr': 1}
{u'-e': 12, u'-ir': 7}
{u'-e': 141, u'E.-6.e': 107}
{u'i-je': 8, u'-r': 30}
{u'-je': 23, u'-r': 52}
{u'-e': 11, u'-vir': 3}
{u'i-je': 150, u'-r': 193, u'-je': 19}
{u'-je': 6, u'wa-6vwar': 8, u'-r': 2}
{u'-He': 48, u'-r': 2}
{u'-tr': 9, u'-je': 11, u'E-avwar': 1, u'-r': 14}
{u'-

{u'irE-je': 8, u'rE-': 34}
{u'irE-y': 2, u'rE-t': 6, u'irE-je': 11, u'rE-': 83}
{u'irE-y': 3, u'rE-je': 10, u'irE-je': 9, u'rE-': 23}

(u'ai3P', u'ii1P')
{u'r-zj\xf4': 3, u'ir-6zj\xf4': 4, u'r-sj\xf4': 1}
{u'ir-j\xf4': 2, u'r-sj\xf4': 3}
{u'r-zj\xf4': 5, u'ir-j\xf4': 94, u'r-sj\xf4': 86}
{u'yr-uvj\xf4': 2, u'yr-Esj\xf4': 1}
{u'ir-j\xf4': 1, u'r-sj\xf4': 22}
{u'r-sj\xf4': 1, u'r-\xf4': 9}
{u'yr-avj\xf4': 2, u'yr-6vj\xf4': 7}
{u'Er-j\xf4': 9, u'Er-ij\xf4': 188}
{u'Er-j\xf4': 19, u'Er-\xf4': 5}
{u'yr-waj\xf4': 4, u'yr-Esj\xf4': 5}
{u'r-zj\xf4': 5, u'r-j\xf4': 3, u'r-sj\xf4': 34}
{u'ir-Etj\xf4': 8, u'ir-j\xf4': 2}

(u'pI2S', u'ppFP')
{u'-e': 11, u'-vi': 3}
{u'-je': 3, '-': 15, u'-t': 23}
{u'i-je': 138, u'-je': 9, '-': 143, u'-t': 18}
{u'i-y': 2, u'i-je': 8, '-': 23, u'-je': 10}
{u'-t': 13, u'E-y': 8, u'-je': 17}
{u'-e': 220, u'-dy': 7, u'-ti': 2}
{u'-e': 5, u'r-Ert': 9}
{u'i-je': 3, u'-t': 2}
{u'-dy': 25, u'-ti': 4}
{u'-e': 28, u'-ti': 5}
{u'-e': 113, u'-dy': 1}
{u'i-y': 1, u'i-je': 1}
{u'

{u'dr\xf4-': 27, u'r\xf4-y': 8}
{u'r\xf4-': 33, u'ir\xf4-je': 8}

(u'is1P', u'is1S')
zéro {u'j\xf4-': 11}

(u'ppMP', u'pi2P')
{u'i-je': 2, u'-se': 27, u'-ve': 10}
{u'y-ize': 1, u'y-e': 5, u'y-Eze': 1}
{u'-t': 4, u'-se': 52, u'i-e': 25}
{u'-ze': 1, u'-se': 4, u'i-e': 1}
{u'y-waje': 5, u'y-e': 8}
{u'i-waje': 1, u'i-Eje': 1}
{u'-t': 6, u'-se': 78}
{u'-se': 2, u'i-Ete': 4, u'i-e': 3}
{u'-t': 4, u'-se': 7, u'-ze': 18}
{u'y-Eze': 1, u'y-6ve': 1}
{u'i-waje': 1, u'i-Eje': 1}
{u'-se': 2, u'i-e': 6}
{u'y-ize': 2, u'y-Eze': 1}

(u'ai2S', u'ai2P')
zéro {u'-t': 41}

(u'pP', u'ii3P')
{u'Ej\xe2-avE': 2, u'\xe2-E': 44}

(u'ii1P', u'ii3P')
{u'ij\xf4-E': 118, u'\xf4-E': 7}
{u'ij\xf4-jE': 7, u'\xf4-E': 2}

(u'pI1P', u'pi2S')
{u'O.\xf4-o.': 43, u'\xf4-': 96}
{u'\xf4-': 225, u't\xf4-': 38}
{u'al\xf4-va': 3, u'\xf4-': 9}
{u'j\xf4-': 13, u'\xf4-': 29}
{u'j\xf4-': 21, u'\xf4-': 103}
{u'O.\xf4-o.': 4, u'\xf4-': 7}
{u'z\xf4-': 10, u'\xf4-': 55}
{u'\xf4-': 34, u'd\xf4-': 41}
{u's\xf4-': 231, u'\xf4-': 54}
{u'\xf

{u'-sE': 4, u'i-E': 6}
{u'-zE': 5, u'-sE': 53, u'i-E': 20}
{u'-sE': 2, u'i-EtE': 12}
{u'y-uvE': 1, u'y-E': 3}
{u'-zE': 23, u'-sE': 8, u'-jE': 2}
{u'y-EzE': 1, u'y-6vE': 1}
{u'-sE': 6, u'i-E': 6}
{u'e-E': 424, u'e-EsE': 1}
{u'y-E': 3, u'y-6vE': 7}

(u'ii1P', u'fi2P')
{u'zj\xf4-re': 36, u'j\xf4-6re': 158}
{u'6nj\xf4-j\xeadre': 26, u'6.j\xf4-E.6re': 1}
{u'j\xf4-re': 23, u'j\xf4-6re': 1617}
{u'zj\xf4-re': 5, u'j\xf4-6re': 5}
{u'j\xf4-re': 3, u'waj\xf4-Ere': 13}
{u'j\xf4-re': 58, u'j\xf4-ire': 23, u'j\xf4-6re': 351}
{u'EJ\xf4-\xeadre': 20, u'\xf4-6re': 1}
{u'alj\xf4-ire': 3, u'j\xf4-6re': 40}
{u'j\xf4-ire': 4, u'j\xf4-6re': 81}
{u'6.j\xf4-E.6re': 29, u'j\xf4-6re': 2}
{u'j\xf4-re': 19, u'\xf4-6re': 16}
{u'j\xf4-6re': 26, u'sj\xf4-tre': 9}
{u'vj\xf4-re': 12, u'j\xf4-re': 2, u'j\xf4-6re': 12}
{u'j\xf4-6re': 22, u'sj\xf4-re': 217}
{u'j\xf4-re': 7, u'ij\xf4-6re': 188}

(u'ppFP', u'ai3P')
{u't-zir': 18, u't-r': 4}
{u'y-ir': 58, u'-r': 8}

(u'ps1P', u'pi3S')
{u'j\xf4-': 2, u'asj\xf4-E': 7}
{u'j\xf

{u'-E': 21, u'E.-6.E': 43}
{u'-jE': 1, u'wa-y': 6}
{u'-E': 1, u'z-': 5}
{u'-jE': 7, u'wa-i': 7, u'wa-y': 1}

(u'pc2S', u'pi3S')
{u'rE-j': 8, u'rE-': 9}
{u'ErE-e': 2, u'rE-j': 7, u'rE-': 12}
{u'tirE-': 15, u'rE-': 20}
{u'6rE-': 4, u'O.6rE-o.': 3}
{u'irE-va': 3, u'rE-': 441}
{u'ErE-e': 5, u'rE-j': 3, u'ErE-wa': 13, u'rE-': 9}
{u'6rE-': 28, u'6rE-E': 3}
{u'6rE-E': 2, u'6.6rE-9.': 1}
{u'irE-': 12, u'rE-': 24}
{u'ErE-e': 2, u'jErE-wa': 2}
{u'j6rE-': 18, u'6rE-': 34}
{u'6rE-': 487, u'O.6rE-o.': 73}

(u'pi1S', u'ai2P')
zéro {u'E-it': 7}

(u'ii1S', u'ppFP')
{u'E-e': 904, u'E-y': 60, u'E-i': 18}
{u'E-e': 1257, u'E-y': 37}
{u'E-e': 17, u'sE-': 190}
{u'E-e': 403, u'E-i': 15}
{u'E-e': 5, u'rE-Ert': 9}
{u'E-e': 2, u'EsE-y': 7}
{u'E-e': 2, u'ivE-Eky': 3}
{u'E-e': 2, u'sE-': 38}
{u'E-e': 15, u'wajE-y': 12}
{u'E-e': 10, u'jE-t': 6}
{u'E-e': 7, u'avE-y': 2}
{u'E-e': 111, u'zE-t': 33}
{u'E-e': 4, u'avE-y': 1}
{u'E-e': 7, u'sE-': 23}
{u'E-e': 79, u'izE-y': 3}
{u'E-e': 1, u'EjE-iz': 2}
{u'E-e': 11, u'EJE-

In [116]:
zeroPop={}
for paire in zeroEntropie:
    for t in rules[paire].classe:
        pop=0
        for c in rules[paire].classe[t]:
            pop+=rules[paire].classe[t][c]
        zeroPop[paire]=pop
    

In [117]:
stemspaceThresh=50
stemspaceRules=[]
for k in sorted(zeroPop, key=zeroPop.get,reverse=True):
    print k, zeroPop[k]
    if zeroPop[k]>=stemspaceThresh:
        stemspaceRules.append(k)

(u'ii3S', u'ii3P') 4545
(u'ii1S', u'ii3S') 4545
(u'ii3P', u'ii2S') 4545
(u'ii3P', u'ii3S') 4545
(u'ppMP', u'ppMS') 4545
(u'ii1S', u'ii2S') 4545
(u'ii2S', u'ii3P') 4545
(u'ii3P', u'ii1S') 4545
(u'ii1S', u'ii3P') 4545
(u'ii3S', u'ii1S') 4545
(u'ii2S', u'ii1S') 4545
(u'ppMS', u'ppMP') 4545
(u'ii3S', u'ii2S') 4545
(u'ii2S', u'ii3S') 4545
(u'ppFS', u'ppFP') 4518
(u'ppFP', u'ppFS') 4518
(u'ai3S', u'ai2S') 4503
(u'is3S', u'ai3S') 4503
(u'ai3S', u'is3S') 4503
(u'ai2S', u'is3S') 4503
(u'ai2S', u'ai3S') 4503
(u'is3S', u'ai2S') 4503
(u'ppFP', u'ppMS') 4384
(u'ppFP', u'ppMP') 4384
(u'ppFS', u'ppMS') 4384
(u'ppFS', u'ppMP') 4384
(u'pP', u'pI1P') 4232
(u'ai3P', u'ai1S') 4090
(u'ai1S', u'ai3P') 4090
(u'ai1P', u'ai3S') 3782
(u'ai3S', u'ai1P') 3782
(u'ai1P', u'ai2S') 3782
(u'ai2S', u'ai1P') 3782
(u'is3S', u'ai1P') 3782
(u'ai1P', u'is3S') 3782
(u'ai1P', u'ai1S') 3398
(u'ai1S', u'ai1P') 3398
(u'pc1P', u'pc2P') 2991
(u'ii1P', u'ps2P') 2342
(u'ps1P', u'ps2P') 2086
(u'ps2P', u'ps1P') 2086
(u'is3P', u'is1S')

In [118]:
len(stemspaceRules)

108

In [119]:
stemSpace=nx.Graph()
for c1,c2 in stemspaceRules:
    if (c2,c1) in stemspaceRules:
#        print c1,c2
        stemSpace.add_edge(c1,c2)
stemSpaceCliques=list(nx.find_cliques(stemSpace))
len(stemSpaceCliques),sorted(stemSpaceCliques,key=len,reverse=True)

(6,
 [[u'is1S',
   u'is2S',
   u'ai3S',
   u'ai3P',
   u'ai1P',
   u'ai1S',
   u'ai2S',
   u'is3P',
   u'is3S'],
  [u'ii1S', u'ii3S', u'ii2S', u'ii3P'],
  [u'ppFS', u'ppFP'],
  [u'pc1P', u'pc2P'],
  [u'ppMS', u'ppMP'],
  [u'ps1P', u'ps2P']])

In [120]:
fSamplePMS="paradigmes.csv"
fSamplePMO="Morphomes-paradigmes.csv"

In [121]:
def openSample(numero,typeSample="PMS"):
    result=None
    fSample=""
    if typeSample=="PMS" and numero in samples:
        fSample=rep+samples[numero]+fSamplePMS
    elif typeSample=="PMO" and numero in samples:
        fSample=rep+samples[numero]+fSamplePMO
    if fSample:
        result=pd.read_csv(rep+samples[numero]+fSamplePMS,sep=";",index_col=0,keep_default_na=False,encoding="utf8")
    return result    

In [122]:
df=openSample(2,typeRules)

In [123]:
def makeZeros(sampleFormes,inFormes=True,contextFree=True):
    series=sampleFormes.copy()
    series={k:v for k,v in series.iteritems() if v!=""}

    paires=rules.keys()
    formesC={}
    for c in series:
        if not c in formesC:
            formesC[c]=set()
        formesC[c].add(series[c])
        if debug: print c, series[c]
        formeC=series[c]
        pairesC=[(c1,c2) for (c1,c2) in paires if c1==c]
        for p in pairesC:
            formeP=rules[p].sortirForme(series[c],contextFree=contextFree)
            if formeP and len(formeP)==1:
                p1=p[1]
                if debug: print p1,formeP
                if p1 not in formesC:
                    formesC[p1]=set()
                formesC[p1].add(formeP.keys()[0])
    if inFormes:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1}
    else:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1 and k not in series}

    return formesZero

In [124]:
def fillStemSpace(sampleFormes,inFormes=True,contextFree=True):
    series=sampleFormes.copy()
    series={k:v for k,v in series.iteritems() if v!=""}

    paires=stemspaceRules
    formesC={}
    for c in series:
        if not c in formesC:
            formesC[c]=set()
        formesC[c].add(series[c])
        if debug: print c, series[c]
        formeC=series[c]
        pairesC=[(c1,c2) for (c1,c2) in paires if c1==c]
        for p in pairesC:
            formeP=rules[p].sortirForme(series[c],contextFree=contextFree)
            if formeP and len(formeP)==1:
                p1=p[1]
                if debug: print p1,formeP
                if p1 not in formesC:
                    formesC[p1]=set()
                formesC[p1].add(formeP.keys()[0])
    if inFormes:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1}
    else:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1 and k not in series}

    return formesZero

In [125]:
cases=[c for c in df.columns.tolist() if c!="lexeme"]
cases=sorted(cases)

In [126]:
lexemes=u"clore éclore falloir valoir".split(" ")
lexemes=[l for l in df["lexeme"].tolist() if not l.endswith(u"er")]
lexemes

[u'abasourdir',
 u'abattre',
 u'abolir',
 u'aboutir',
 u'abrutir',
 u'absoudre',
 u'abstenir',
 u'abstraire',
 u'ab\xe2tardir',
 u'ab\xeatir',
 u'accomplir',
 u'accourcir',
 u'accourir',
 u'accroire',
 u'accroupir',
 u'accro\xeetre',
 u'accueillir',
 u'acqu\xe9rir',
 u'adjoindre',
 u'admettre',
 u'adoucir',
 u'advenir',
 u'affadir',
 u'affaiblir',
 u'affermir',
 u'affranchir',
 u'agir',
 u'agonir',
 u'agrandir',
 u'aguerrir',
 u'ahurir',
 u'aigrir',
 u'alanguir',
 u'alentir',
 u'alourdir',
 u'alunir',
 u'amaigrir',
 u'amerrir',
 u'amincir',
 u'amoindrir',
 u'amollir',
 u'amortir',
 u'anoblir',
 u'an\xe9antir',
 u'apercevoir',
 u'aplanir',
 u'aplatir',
 u'appara\xeetre',
 u'appartenir',
 u'appauvrir',
 u'appendre',
 u'appesantir',
 u'applaudir',
 u'apprendre',
 u'approfondir',
 u'arrondir',
 u'assagir',
 u'assaillir',
 u'assavoir',
 u'asseoir',
 u'asservir',
 u'assombrir',
 u'assortir',
 u'assoupir',
 u'assouplir',
 u'assourdir',
 u'assouvir',
 u'assujettir',
 u'astreindre',
 u'atteindr

In [127]:
dfNew=df.copy()
for lexeme in lexemes:
    index=df.loc[df['lexeme']==lexeme].index.values
    if index:
        i=index[0]
        sampleFormes=df.iloc[i].to_dict()
        sampleFormes={k:v for k,v in sampleFormes.iteritems() if v!=""}
        lexemeI=sampleFormes["lexeme"]
        print lexemeI,
#        formesZero=makeZeros(sampleFormes,inFormes=False)
        formesZero=fillStemSpace(sampleFormes,inFormes=False,contextFree=not avecContexte)
        for case in cases:
            if case in sampleFormes:
#                print "\t",case,"\t  ",sampleFormes[case]
                continue
            elif case in formesZero:
#                print "\t",case,"\t=>",formesZero[case]
                dfNew.iloc[i][case]=formesZero[case]
            else:
#                print "\t",case
                continue
#        print

abasourdir abattre abolir aboutir abrutir absoudre abstenir abstraire abâtardir abêtir accomplir accourcir accourir accroire accroupir accroître accueillir acquérir adjoindre admettre adoucir advenir affadir affaiblir affermir affranchir agir agonir agrandir aguerrir ahurir aigrir alanguir alentir alourdir alunir amaigrir amerrir amincir amoindrir amollir amortir anoblir anéantir apercevoir aplanir aplatir apparaître appartenir appauvrir appendre appesantir applaudir apprendre approfondir arrondir assagir assaillir assavoir asseoir asservir assombrir assortir assoupir assouplir assourdir assouvir assujettir astreindre atteindre attendre attendrir atterrir attiédir avachir avertir avilir avoir bannir barrir battre blanchir bleuir blondir blottir blêmir boire bondir bouffir bouillir brandir bruire brunir bâtir bénir calmir candir ceindre chancir choir choisir chérir circoncire circonscrire circonvenir clapir clore combattre commettre comparaître compatir complaire comprendre compromettre

## Ouvrir les fichiers de morphomes

In [200]:
morphomeCases=pd.read_pickle(rep+samples[54]+"Morphomes.pkl")[["case","morphome"]].drop_duplicates().to_dict()
casesMC=morphomeCases["case"]
morphomesMC=morphomeCases["morphome"]
listMorphomes=[]
for element in casesMC:
    listMorphomes.append(morphomesMC[element].split("/"))
cases

[u'ai1P',
 u'ai1S',
 u'ai2P',
 u'ai2S',
 u'ai3P',
 u'ai3S',
 u'fi1P',
 u'fi1S',
 u'fi2P',
 u'fi2S',
 u'fi3P',
 u'fi3S',
 u'ii1P',
 u'ii1S',
 u'ii2P',
 u'ii2S',
 u'ii3P',
 u'ii3S',
 u'inf',
 u'is1P',
 u'is1S',
 u'is2P',
 u'is2S',
 u'is3P',
 u'is3S',
 u'pI1P',
 u'pI2P',
 u'pI2S',
 u'pP',
 u'pc1P',
 u'pc1S',
 u'pc2P',
 u'pc2S',
 u'pc3P',
 u'pc3S',
 u'pi1P',
 u'pi1S',
 u'pi2P',
 u'pi2S',
 u'pi3P',
 u'pi3S',
 u'ppFP',
 u'ppFS',
 u'ppMP',
 u'ppMS',
 u'ps1P',
 u'ps1S',
 u'ps2P',
 u'ps2S',
 u'ps3P',
 u'ps3S']

In [196]:
def makeTabularParadigme(lexeme,lDF,dictColours,title="",coulLim=False, cat="V",dictMorphomes={}):
    row=lDF[lDF["lexeme"]==lexeme]
    tabular=[]
    def makeValue(case):
        print case,row[case],len(row[case]),row[case].values
        if len(row[case])>0 and len(row[case].values[0])>0:
            result=coderIPA(row[case].values[0])
        elif dictMorphomes!={} and case in dictMorphomes:
            altCase=dictMorphomes[case][0]
            if len(row[altCase])>0 and len(row[altCase].values[0])>0:
                result=coderIPA(row[altCase].values[0])
            else:
                result="---"
        else:
            result="---"
    def makeLine6(tenseCode):
        line=[tabTemps[tenseCode]]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            case=tenseCode+person
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
        return r" & ".join(line)+r"\\"

    def makeLine3(tenseCode):
        line=[tabTemps[tenseCode]]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            if person in ["2S","1P","2P"]:
                case=tenseCode+person
                if case in dictColours:
                    line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
                else:
                    line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
#                line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
            else:
                line.append(r"---")
        return r" & ".join(line)+r"\\"
    
    def makeLineNF():
        line=["non-fini"]
        for case in ["inf","pP","ppMS","ppMP","ppFS","ppFP"]:
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
#            line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
        return r" & ".join(line)+r"\\"

    def makeLineMF(nombre):
        line=[]
        for genre in "mf":
            case=genre+nombre
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],makeValue(case)))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",makeValue(case)))
        return r" & ".join(line)+r"\\"
    
    def makeLineCoulLim():
        line=[]
        for numLimite,limite in enumerate(listLimites):
            line.append(r"\cellcolor{%s}%s"%(listLimCoul[numLimite],"$<$"+str(limite)))
        return r"\hline\hline "+r" & ".join(line)+r"\\"
        
    if cat=="V":
        top=[
            r"\begin{center}",
            r"\begin{tabular}{ccccccc}",
            r"\toprule",
            " & ".join([lexeme]+[p+n for n in ["sg","pl"] for p in "123" ])+r"\\",
            r"\midrule"
            ]
        bottom=[
            r"\bottomrule",
            r"\end{tabular}\\",
            title,
            r"\end{center}",
            r"\bigskip",
            r""
            ]
        tabular.append("\n".join(top))
        for tenseCode in ["pi","ii","fi","pc", "ps","ai", "is"]:
            tabular.append(makeLine6(tenseCode))
        tabular.append(makeLine3("pI"))
        tabular.append(makeLineNF())
    elif cat=="A":
        top=[
            r"\begin{center}",
            r"\begin{tabular}{cc}",
            r"\toprule"
            ]
        bottom=[
            r"\bottomrule",
            r"\end{tabular}\\",
            title,
            r"\end{center}",
            r"\bigskip",
            r""
            ]
        tabular.append("\n".join(top))
        for number in "sp":
            tabular.append(makeLineMF(number))
    if coulLim:
        tabular.append(makeLineCoulLim())
    tabular.append("\n".join(bottom))
    return "\n".join(tabular)

In [162]:
etColours=["orange",
           "brown!50",
           "brown",
           "blue!10",
           "yellow!50",
           "blue!25",
           "teal!50",
           "blue!50",
           "cyan!50",
           "lime",
           "pink",
           "magenta!50",
          ]
etCells=[
    ["pi1P","pi2P"]+["ii"+p+n for p in "123" for n in "SP"],
    ["pi3P"],
    ["pi"+p+"S" for p in "123"],
    ["pP"],
    ["pI2S"],
    ["pI1P","pI2P"],
    ["ps"+p+"S" for p in "123"]+["ps3P"],
    ["ps1P","ps2P"],
    ["inf"],
    ["fi"+p+n for p in "123" for n in "SP"]+["pc"+p+n for p in "123" for n in "SP"],
    ["ai"+p+n for p in "123" for n in "SP"]+["is"+p+n for p in "123" for n in "SP"],
    ["pp"+g+n for g in "MF" for n in "SP"],
        ]
tabTemps={
    "pi":u"ind. prs",
    "ii":u"ind. ipf",
    "ai":u"ind. ps",
    "fi":u"ind. fut",
    "ps":u"subj. prs",
    "is":u"subj. ipf",
    "pc":u"cond. prs",
    "pI":u"imper. prs",
    "inf":u"non-fini"
    }
dictEtColours={}
for nET,ET in enumerate(etCells):
    for c in ET:
        dictEtColours[c]=etColours[nET]
dictEtColours

{'ai1P': 'pink',
 'ai1S': 'pink',
 'ai2P': 'pink',
 'ai2S': 'pink',
 'ai3P': 'pink',
 'ai3S': 'pink',
 'fi1P': 'lime',
 'fi1S': 'lime',
 'fi2P': 'lime',
 'fi2S': 'lime',
 'fi3P': 'lime',
 'fi3S': 'lime',
 'ii1P': 'orange',
 'ii1S': 'orange',
 'ii2P': 'orange',
 'ii2S': 'orange',
 'ii3P': 'orange',
 'ii3S': 'orange',
 'inf': 'cyan!50',
 'is1P': 'pink',
 'is1S': 'pink',
 'is2P': 'pink',
 'is2S': 'pink',
 'is3P': 'pink',
 'is3S': 'pink',
 'pI1P': 'blue!25',
 'pI2P': 'blue!25',
 'pI2S': 'yellow!50',
 'pP': 'blue!10',
 'pc1P': 'lime',
 'pc1S': 'lime',
 'pc2P': 'lime',
 'pc2S': 'lime',
 'pc3P': 'lime',
 'pc3S': 'lime',
 'pi1P': 'orange',
 'pi1S': 'brown',
 'pi2P': 'orange',
 'pi2S': 'brown',
 'pi3P': 'brown!50',
 'pi3S': 'brown',
 'ppFP': 'magenta!50',
 'ppFS': 'magenta!50',
 'ppMP': 'magenta!50',
 'ppMS': 'magenta!50',
 'ps1P': 'blue!50',
 'ps1S': 'teal!50',
 'ps2P': 'blue!50',
 'ps2S': 'teal!50',
 'ps3P': 'teal!50',
 'ps3S': 'teal!50'}

In [197]:
print makeTabularParadigme(u"clore",dfNew,dictEtColours,dictMorphomes=dictCaseMorphome)

pi1S 775    klo
Name: pi1S, dtype: object 1 [u'klo']
pi2S 775    
Name: pi2S, dtype: object 1 [u'']
pi3S 775    klo
Name: pi3S, dtype: object 1 [u'klo']
pi1P 775    
Name: pi1P, dtype: object 1 [u'']
pi2P 775    
Name: pi2P, dtype: object 1 [u'']
pi3P 775    
Name: pi3P, dtype: object 1 [u'']
ii1S 775    
Name: ii1S, dtype: object 1 [u'']
ii2S 775    
Name: ii2S, dtype: object 1 [u'']
ii3S 775    
Name: ii3S, dtype: object 1 [u'']
ii1P 775    
Name: ii1P, dtype: object 1 [u'']
ii2P 775    
Name: ii2P, dtype: object 1 [u'']
ii3P 775    
Name: ii3P, dtype: object 1 [u'']
fi1S 775    
Name: fi1S, dtype: object 1 [u'']
fi2S 775    
Name: fi2S, dtype: object 1 [u'']
fi3S 775    
Name: fi3S, dtype: object 1 [u'']
fi1P 775    
Name: fi1P, dtype: object 1 [u'']
fi2P 775    
Name: fi2P, dtype: object 1 [u'']
fi3P 775    
Name: fi3P, dtype: object 1 [u'']
pc1S 775    
Name: pc1S, dtype: object 1 [u'']
pc2S 775    
Name: pc2S, dtype: object 1 [u'']
pc3S 775    
Name: pc3S, dtype: object 1 [u'']
p

In [134]:
lexeme=u"extraire"
ix=df.loc[df.lexeme==lexeme,:].T.columns.values[0]
pd.concat([df.loc[df.lexeme==lexeme,:].T.rename(columns={ix:'Old'}),
dfNew.loc[dfNew.lexeme==lexeme,:].T.rename(columns={ix:'New'})],
         axis=1)

,Old,New
lexeme,extraire,extraire
ai1P,,
ai1S,,
ai2P,,
ai2S,,
ai3P,,
ai3S,,
fi1P,,
fi1S,,
fi2P,,


In [59]:
dfNew.loc[dfNew.lexeme==u"valoir",:].T

,4262
lexeme,valoir
ai1P,valym
ai1S,valy
ai2P,
ai2S,valy
ai3P,valyr
ai3S,valy
fi1P,
fi1S,
fi2P,


In [42]:
for i in range(500):
    sampleFormes=df.loc[i].to_dict()
    lexemeI=sampleFormes["lexeme"]
    if not lexemeI.endswith("er"):
        print lexemeI
        numero=1
        for k,v in sampleFormes.iteritems():
            if k!="lexeme" and v!="":
                print numero,k,v
                numero+=1
        print
        formesZero=makeZeros(sampleFormes,inFormes=False)
        for k,v in formesZero.iteritems():
            if k!="lexeme":
                print numero,k,v
                numero+=1
        print

abasourdir
1 pP abazurdisâ
2 ai3S abazurdi
3 ai3P abazurdir
4 ps3S abazurdis
5 ii3S abazurdisE
6 inf abazurdir
7 ii3P abazurdisE
8 pi2S abazurdi
9 pi2P abazurdise
10 ppFS abazurdi
11 pI2S abazurdi
12 fi2P abazurdire
13 pi1S abazurdi
14 pi3P abazurdis
15 pi3S abazurdi
16 ppMS abazurdi
17 pc3S abazurdirE
18 pc3P abazurdirE
19 fi3S abazurdira
20 fi3P abazurdirô
21 ppMP abazurdi
22 fi1S abazurdirE
23 pc1S abazurdirE

24 pI1P abazurdisô
25 ii1S abazurdisE
26 ii2S abazurdisE
27 pi1P abazurdisô
28 ps2P abazurdje
29 ppFP abazurdi
30 is2S abazurdis
31 pc1P abazurdirjô
32 is1S abazurdis
33 ai2S abazurdi
34 ps2S abazurdis
35 is3P abazurdis
36 is3S abazurdi
37 ai1P abazurdim
38 ai1S abazurdi
39 pI2P abazurdise
40 ps3P abazurdis
41 pc2P abazurdirje
42 ps1S abazurdis

abattre
1 ii1P abatjô
2 ii1S abatE
3 pP abatâ
4 ai3S abati
5 ai3P abatir
6 ps3S abat
7 ii3S abatE
8 inf abatr
9 ii3P abatE
10 pi2S aba
11 pi2P abate
12 ppFS abaty
13 ppFP abaty
14 fi2P abatre
15 pI2P abate
16 ps2S abat
17 ii2P abatje
1

2 ii1S admEtE
3 pP admEtâ
4 ai3S admi
5 ps3S admEt
6 ii3S admEtE
7 inf admEtr
8 ii3P admEtE
9 pi2S admE
10 ai1S admi
11 pi2P admEte
12 ppFS admiz
13 ppFP admiz
14 pI2S admE
15 fi2P admEtre
16 fi2S admEtra
17 pI2P admEte
18 ps2S admEt
19 ii2P admEtje
20 pi1S admE
21 pi1P admEtô
22 pi3P admEt
23 pi3S admE
24 ppMS admi
25 pc3S admEtrE
26 pc3P admEtrE
27 fi3S admEtra
28 fi3P admEtrô
29 ppMP admi
30 fi1P admEtrô
31 fi1S admEtrE
32 pc1S admEtrE
33 pI1P admEtô

34 ps2P admEsje
35 ai3P admir
36 ai2S admi
37 pc2P admâdrije
38 is3S admi
39 ai1P admim
40 ps3P admEt
41 ii2S admEtE
42 ps1S admEt

adoucir
1 ii1S adusisE
2 pP adusisâ
3 ai3S adusi
4 ai3P adusir
5 ps3S adusis
6 ii3S adusisE
7 inf adusir
8 ii3P adusisE
9 pi2S adusi
10 pi2P adusise
11 ppFS adusi
12 ppFP adusi
13 pI2S adusi
14 fi2P adusire
15 fi2S adusira
16 pi1S adusi
17 pi3P adusis
18 pi3S adusi
19 ppMS adusi
20 pc3S adusirE
21 pc3P adusirE
22 fi3S adusira
23 fi3P adusirô
24 ppMP adusi
25 fi1P adusirô
26 fi1S adusirE
27 pc1S adusirE

28

42 ps1S amEgris

amerrir
1 pP amErjâ
2 ai3S amErja
3 ai3P amErjEr
4 ii3S amErjE
5 inf amErje
6 ii3P amErjE
7 pi2S amEri
8 ai1S amErjE
9 pi2P amErje
10 ppFS amErje
11 ppFP amErje
12 pI2S amEri
13 fi2P amErire
14 fi2S amErira
15 pi1S amEri
16 pi1P amErjô
17 pi3P amEri
18 pi3S amEri
19 ppMS amErje
20 pc3S amErirE
21 fi3S amErira
22 ppMP amErje
23 fi1P amErirô
24 fi1S amErirE
25 pc1S amErirE

26 ii1S amErjE
27 ii2S amErjE
28 ps2S amEri
29 is2S amErjas
30 is1S amErjas
31 pc1P amErrjô
32 ai2S amErja
33 is3P amErjas
34 is3S amErja
35 ai1P amErjam
36 ps3S amEri
37 ps3P amEri
38 pI2P amErje
39 pI1P amErjô
40 ps1S amEri

amincir
1 ii1S amêsisE
2 pP amêsisâ
3 ai3S amêsi
4 ai3P amêsir
5 ps3S amêsis
6 ii3S amêsisE
7 inf amêsir
8 ii3P amêsisE
9 pi2S amêsi
10 pi2P amêsise
11 ppFS amêsi
12 ppFP amêsi
13 pI2S amêsi
14 fi2P amêsire
15 pi1S amêsi
16 pi3P amêsis
17 pi3S amêsi
18 ppMS amêsi
19 pc3S amêsirE
20 pc3P amêsirE
21 fi3S amêsira
22 fi3P amêsirô
23 ppMP amêsi
24 fi1S amêsirE
25 pc1S amêsirE

26 pI1

26 pi1P aprOfôdisô
27 ps2P aprOfôdje
28 ps2S aprOfôdis
29 is2S aprOfôdis
30 pc1P aprOfôdirjô
31 ai3S aprOfôdi
32 is1S aprOfôdis
33 ai2S aprOfôdi
34 is3P aprOfôdis
35 is3S aprOfôdi
36 ai1P aprOfôdim
37 ai1S aprOfôdi
38 pI2P aprOfôdise
39 ps3P aprOfôdis
40 pc2P aprOfôdirje
41 ps1S aprOfôdis

arrondir
1 pP arôdisâ
2 ai3S arôdi
3 ai3P arôdir
4 ps3S arôdis
5 ii3S arôdisE
6 inf arôdir
7 ii3P arôdisE
8 pi2S arôdi
9 ppFS arôdi
10 ppFP arôdi
11 fi2P arôdire
12 fi2S arôdira
13 pi1S arôdi
14 pi3P arôdis
15 pi3S arôdi
16 ppMS arôdi
17 pc3S arôdirE
18 fi3S arôdira
19 fi3P arôdirô
20 ppMP arôdi
21 fi1P arôdirô
22 fi1S arôdirE

23 pI1P arôdisô
24 ii1S arôdisE
25 ii2S arôdisE
26 pi1P arôdisô
27 ps2P arôdje
28 ps2S arôdis
29 is2S arôdis
30 pc1P arôdirjô
31 is1S arôdis
32 ai2S arôdi
33 is3P arôdis
34 is3S arôdi
35 ai1P arôdim
36 ai1S arôdi
37 pI2P arôdise
38 pI2S arôdi
39 ps3P arôdis
40 pc2P arôdirje
41 ps1S arôdis

assagir
1 ii1P asaZisjô
2 ii1S asaZisE
3 pP asaZisâ
4 ai3S asaZi
5 ai3P asaZir
6 is3S as

13 pi3S atâdri
14 ppMS atâdri
15 pc3S atâdrirE
16 pc3P atâdrirE
17 ppMP atâdri

18 pI1P atâdrisô
19 ii2S atâdrisE
20 pi1P atâdrisô
21 ps2P atâdrje
22 ppFP atâdri
23 is2S atâdris
24 is1S atâdris
25 pc1P atâdrirjô
26 ai2S atâdri
27 ps2S atâdris
28 is3P atâdris
29 is3S atâdri
30 ps3S atâdris
31 ai1P atâdrim
32 ps3P atâdris
33 pI2P atâdrise
34 pI2S atâdri
35 ai1S atâdri
36 pc2P atâdrirje
37 ps1S atâdris

atterrir
1 pP atErisâ
2 ai3S atEri
3 ai3P atErir
4 ps3S atEris
5 ai1P atErim
6 inf atErir
7 ii3P atErisE
8 pi2S atEri
9 fi2P atErire
10 fi2S atErira
11 pi1S atEri
12 pi3P atEris
13 pi3S atEri
14 ppMS atEri
15 pc3S atErirE
16 fi3S atErira
17 fi3P atErirô
18 ppMP atEri
19 fi1P atErirô
20 fi1S atErirE
21 pc1S atErirE

22 pI1P atErisô
23 ii1S atErisE
24 ii2S atErisE
25 pi1P atErisô
26 ps2S atEris
27 is2S atEris
28 is1S atEris
29 ai2S atEri
30 is3P atEris
31 is3S atEri
32 ii3S atErisE
33 ai1S atEri
34 pI2P atErise
35 pI2S atEri
36 ps3P atEris
37 ps1S atEris

attiédir
1 ii1P atjEdisjô
2 ii1S atj

In [ ]:
rules[("pi3S","ii1S")].sortirForme("Xi",contextFree=True)